# EfficientDet Tutorial: inference, eval, and training 

In [19]:
MODEL = 'efficientdet-d0'  #@param
# Prepare image and visualization settings.
img_path = os.path.join(os.getcwd(), 'img.png')
# Get the largest of height/width and round to 128.
image_size = max(PIL.Image.open(img_path).size)
print(image_size)

1920


# 4. Training EfficientDets on PASCAL.

## 4.1 Prepare data

In [24]:
# Get pascal voc 2012 trainval data
#!PYTHONPATH=".:$PYTHONPATH"  python dataset/create_pascal_tfrecord.py --data_dir=VOCdevkit --year=VOC2012  --output_path=tfrecord/pascal

# Pascal has 5717 train images with 100 shards epoch, here we use a single shard
# for demo, but users should use all shards pascal-*-of-00100.tfrecord.
file_pattern = 'pascal-00000-of-00100.tfrecord'  # @param
images_per_epoch = 57 * len(tf.io.gfile.glob('tfrecord/' + file_pattern))
images_per_epoch = images_per_epoch // 8 * 8  # round to 64.
print('images_per_epoch = {}'.format(images_per_epoch))

images_per_epoch = 56


## 4.2 Train Pascal VOC 2012 from ImageNet checkpoint for Backbone.

In [36]:
from os import makedirs
# Train efficientdet from scratch with backbone checkpoint.
backbone_name = {
    'efficientdet-d0': 'efficientnet-b0',
    'efficientdet-d1': 'efficientnet-b1',
    'efficientdet-d2': 'efficientnet-b2',
    'efficientdet-d3': 'efficientnet-b3',
    'efficientdet-d4': 'efficientnet-b4',
    'efficientdet-d5': 'efficientnet-b5',
    'efficientdet-d6': 'efficientnet-b6',
    'efficientdet-d7': 'efficientnet-b6',
}[MODEL]
print('cwd: {}'.format(os.getcwd()))

cwd: /home/xintian/git/automl/efficientdet


In [38]:
# generating train tfrecord is large, so we skip the execution here.
if backbone_name not in os.listdir():
  !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/ckptsaug/{backbone_name}.tar.gz
  !tar xf {backbone_name}.tar.gz
model_saving_dir = os.path.join(os.getcwd(),'tmp/model_dir')
makedirs(model_saving_dir, exist_ok=True)
model_dir = os.path.join(os.getcwd(),'tmp/model_dir/{}-scratch'.format(MODEL))

print('Backbone name: {}'.format(backbone_name))
print('Model saving dir: {}'.format(model_saving_path))
print('Model dir: {}'.format(model_dir))

Backbone name: efficientnet-b0
Model saving dir: /home/xintian/git/automl/efficientdet/tmp/model_dir
Model dir: /home/xintian/git/automl/efficientdet/tmp/model_dir/efficientdet-d0-scratch


In [39]:
# key option: use --backbone_ckpt rather than --ckpt.
# Don't use ema since we only train a few steps.
!python main.py --mode=train_and_eval \
    --training_file_pattern=tfrecord/{file_pattern} \
    --validation_file_pattern=tfrecord/{file_pattern} \
    --model_name={MODEL} \
    --model_dir=/home/xintian/git/automl/efficientdet/tmp/model_dir/{MODEL}-scratch \
    --backbone_ckpt={backbone_name} \
    --train_batch_size=4 \
    --eval_batch_size=4 --eval_samples={images_per_epoch}  \
    --num_examples_per_epoch={images_per_epoch}  --num_epochs=1  \
    --hparams="num_classes=20,moving_average_decay=0,mixed_precision=true"

-b0/blocks_11/se/conv2d/kernel from ckpt var efficientnet-b0/blocks_11/se/conv2d/kernel
I0702 19:18:13.891789 140573777909568 utils.py:123] Init efficientnet-b0/blocks_11/se/conv2d/bias from ckpt var efficientnet-b0/blocks_11/se/conv2d/bias
I0702 19:18:13.891842 140573777909568 utils.py:123] Init efficientnet-b0/blocks_11/se/conv2d_1/kernel from ckpt var efficientnet-b0/blocks_11/se/conv2d_1/kernel
I0702 19:18:13.891895 140573777909568 utils.py:123] Init efficientnet-b0/blocks_11/se/conv2d_1/bias from ckpt var efficientnet-b0/blocks_11/se/conv2d_1/bias
I0702 19:18:13.891948 140573777909568 utils.py:123] Init efficientnet-b0/blocks_11/conv2d_1/kernel from ckpt var efficientnet-b0/blocks_11/conv2d_1/kernel
I0702 19:18:13.892004 140573777909568 utils.py:123] Init efficientnet-b0/blocks_11/tpu_batch_normalization_2/gamma from ckpt var efficientnet-b0/blocks_11/tpu_batch_normalization_2/gamma
I0702 19:18:13.892058 140573777909568 utils.py:123] Init efficientnet-b0/blocks_11/tpu_batch_normal

## 4.3 Train Pascal VOC 2012 from COCO checkpoint for the whole net.

In [0]:
# generating train tfrecord is large, so we skip the execution here.
import os
if MODEL not in os.listdir():
  !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/{MODEL}.tar.gz
  !tar xf {MODEL}.tar.gz

!mkdir /tmp/model_dir/
# key option: use --ckpt rather than --backbone_ckpt.
!python main.py --mode=train_and_eval \
    --training_file_pattern=tfrecord/{file_pattern} \
    --validation_file_pattern=tfrecord/{file_pattern} \
    --model_name={MODEL} \
    --model_dir=/tmp/model_dir/{MODEL}-finetune \
    --ckpt={MODEL} \
    --train_batch_size=8 \
    --eval_batch_size=8 --eval_samples={images_per_epoch}  \
    --num_examples_per_epoch={images_per_epoch}  --num_epochs=1  \
    --hparams="num_classes=20,moving_average_decay=0,mixed_precision=true"

mkdir: cannot create directory ‘/tmp/model_dir/’: File exists
2020-06-01 05:38:43.365099: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1

W0601 05:38:45.587865 140233166403456 module_wrapper.py:138] From main.py:234: The name tf.estimator.tpu.TPUConfig is deprecated. Please use tf.compat.v1.estimator.tpu.TPUConfig instead.


W0601 05:38:45.588135 140233166403456 module_wrapper.py:138] From main.py:239: The name tf.estimator.tpu.InputPipelineConfig is deprecated. Please use tf.compat.v1.estimator.tpu.InputPipelineConfig instead.


W0601 05:38:45.588366 140233166403456 module_wrapper.py:138] From main.py:247: The name tf.estimator.tpu.RunConfig is deprecated. Please use tf.compat.v1.estimator.tpu.RunConfig instead.

I0601 05:38:45.588604 140233166403456 main.py:262] {'name': 'efficientdet-d0', 'act_type': 'swish', 'image_size': (512, 512), 'target_size': None, 'input_rand_hflip': True, 'train_scale_min': 0.1, 'train_s